In [1]:
import polars as pl
import joblib
from src.data_loader import get_all_data, split_date_lazy
from src.feature_engineering import build_features_from_purchases, build_labels
from src.trainer import train_model
from src import config


In [2]:
# ======================================================================
# 2. LOAD & SPLIT DATA USING CUSTOM UTILITY
# ======================================================================
print("📂 Loading and splitting data...")

# 1. Load dữ liệu
lf_items, lf_users, lf_purchases = get_all_data("data")

# 2. Sử dụng hàm của bạn để chia tập
# Hàm này tự động xử lý cast datetime nếu created_date đang là String
lf_purchase_hist, lf_purchase_recent, lf_purchase_val = split_date_lazy(
    lf_purchases, 
    date_column_name="created_date"
)

# 3. Kiểm tra nhanh kết quả bằng collect_schema (không tốn RAM)
print("✅ Data split completed.")
print(f"   - Hist samples: {lf_purchase_hist.select(pl.len()).collect().item():,}")
print(f"   - Recent samples: {lf_purchase_recent.select(pl.len()).collect().item():,}")
print(f"   - Val samples: {lf_purchase_val.select(pl.len()).collect().item():,}")

📂 Loading and splitting data...
📂 Đang quét dữ liệu từ thư mục: c:\Users\HP\Desktop\my_rec_system\data
✅ Đã load Items
✅ Đã load Users
✅ Đã load Purchases
✅ Data split completed.
   - Hist samples: 29,580,965
   - Recent samples: 3,099,667
   - Val samples: 3,049,193


In [3]:
# ======================================================================
# 3. FEATURE ENGINEERING & LABELING
# ======================================================================

# 1. Tạo Đặc trưng từ quá khứ (Dùng Hist để không bị Leak)
print("🛠 Building features from Hist...")
train_features_df = build_features_from_purchases(lf_purchase_hist, lf_items, lf_users)

# 2. Tạo Nhãn từ giai đoạn tiếp theo (Recent làm Positive)
print("🎯 Building labels from Recent...")
train_label_df = build_labels(
    baseHist=lf_purchase_hist, 
    labelHist=lf_purchase_recent, 
    items=lf_items,
    negative_ratio=1.0  # Bạn có thể tăng lên 2.0 hoặc 3.0 để có nhiều mẫu âm hơn
)

# 3. Kết hợp Features và Labels thành tập Train hoàn chỉnh
print("🔗 Merging Features and Labels...")
train_df = (
    train_label_df
    .select([
        pl.col("customer_id").alias("X_-1"),
        pl.col("item_id").alias("X_0"),
        pl.col("label").alias("Y")
    ])
    .join(train_features_df, on=["X_-1", "X_0"], how="left")
    # Điền 0 cho những cặp (User, Item) không có trong lịch sử (thường là mẫu âm)
    .with_columns(
        pl.all().exclude(["X_-1", "X_0", "Y"]).fill_null(0)
    )
)

# Soi thử dữ liệu
display(train_df.limit(5).collect())

🛠 Building features from Hist...
🎯 Building labels from Recent...
BUILD LABELS
Step 1 — Loading positives...
 → Positives: 2,749,181
Step 3 — Building exclusion map...
Step 4 — Loading popular items...
Step 5 — Sampling negatives...
   Processed 100,000 users...
   Processed 200,000 users...
   Processed 300,000 users...
   Processed 400,000 users...
   Processed 500,000 users...
   Processed 600,000 users...
Step 6 — Finalizing dataset...
Total time: 1326.14s
🔗 Merging Features and Labels...


X_-1,X_0,Y,X_1,X_2,X_3,X_4,X_5
i32,str,i8,f64,f64,f64,f64,f64
7029481,"""2346000000132""",1,0.0,0.0,0.0,0.0,0.0
4777120,"""5435000000002""",1,0.0,0.0,0.0,0.0,0.0
6022658,"""3491000000031""",0,0.0,0.0,0.0,0.0,0.0
1928555,"""0029010010002""",0,0.0,0.0,0.0,0.0,0.0
8031844,"""3511000000022""",0,0.0,0.0,0.0,0.0,0.0


In [4]:
# ======================================================================
# 4. EXECUTE TRAINING
# ======================================================================
from src.trainer import train_model
import joblib
from src import config

# Xác định danh sách feature (khớp với file feature_engineering.py)
feature_cols = ["X_1", "X_2", "X_3", "X_4", "X_5"]

print(f"🚀 Starting training with features: {feature_cols}")

# Gọi hàm từ trainer.py
model_xgb = train_model(train_df, feature_cols)

# Lưu mô hình vào thư mục model/
import os
if not os.path.exists('model'):
    os.makedirs('model')

joblib.dump(model_xgb, config.MODEL_PATH)
print(f"✅ Model trained and saved at: {config.MODEL_PATH}")

🚀 Starting training with features: ['X_1', 'X_2', 'X_3', 'X_4', 'X_5']
Training XGBoost...
✅ Model trained and saved at: models/xgb_model.pkl


In [5]:
# ======================================================================
# 4.5 PREPARE INFERENCE FEATURES (TEST POOL) - FIXED
# ======================================================================
print("🛠 Building features for Jan 2025 Inference...")

# 1. Gộp toàn bộ dữ liệu lịch sử
full_history = pl.concat([lf_purchase_val])

# 2. Tạo đặc trưng (X_1, X_2, X_3)
from src.feature_engineering import build_features_from_purchases

infer_features_df = build_features_from_purchases(
    lf_purchases=full_history, 
    lf_items=lf_items, 
    lf_users=lf_users
)

# SỬA LỖI TẠI ĐÂY: Dùng select và n_unique trong Lazy mode
count_user = (
    infer_features_df
    .select(pl.col("X_-1").n_unique())
    .collect()
    .item()
)

print(f"✅ Created inference features for {count_user:,} users.")

# Soi thử bảng đặc trưng
display(infer_features_df.limit(5).collect())

🛠 Building features for Jan 2025 Inference...
✅ Created inference features for 628,069 users.


X_-1,X_0,X_1,X_2,X_3,X_4,X_5
i32,str,f64,f64,f64,f64,f64
4895259,"""2808000000001""",1.0,8.0,1.0,9.127946,9.124129
4192337,"""0950000000057""",2.0,4.0,1.0,1.273278,2.772589
620605,"""6235000000002""",1.0,1.0,1.0,2.373333,8.004366
3799514,"""2408000000010""",3.0,3.0,3.0,0.585487,6.84588
1304875,"""3286000000004""",1.0,1.0,1.0,1.782017,8.343316


In [ ]:
user_items = (
    full_history
    .select([
        pl.col("customer_id").cast(pl.Int64), 
        pl.col("item_id").cast(pl.Int64)
    ])
    .drop_nulls()
    .unique() 
    .group_by("customer_id")
    .agg(pl.col("item_id").alias("item_id_list"))
    .collect()
)
user_items_clean = (
    user_items
    .explode("item_id_list")
    .rename({"item_id_list": "item_id"})
    .cast({"item_id": pl.Int64})
)

item_pairs = (
    user_items_clean.join(user_items_clean, on="customer_id", suffix="_right")
    .filter(pl.col("item_id") < pl.col("item_id_right"))
    .group_by(["item_id", "item_id_right"])
    .len()
    .filter(pl.col("len") > 2)
)

top_item_links = (
    pl.concat([
        item_pairs.select([pl.col("item_id"), pl.col("item_id_right"), "len"]),
        item_pairs.select([pl.col("item_id_right").alias("item_id"), pl.col("item_id").alias("item_id_right"), "len"])
    ])
    .sort(["item_id", "len"], descending=[False, True])
    .group_by("item_id")
    .head(10)
    .select([
        pl.col("item_id").alias("item_id_source"), 
        pl.col("item_id_right").alias("item_id_rec")
    ])
)

In [ ]:
import xgboost as xgb
import polars as pl
import numpy as np

def run_inference_local_v4(model, infer_features_df, lf_purchases, top_item_links, user_items, top_k=10, batch_size=2000):
    # 1. Candidate Pool: Top 400 món phổ biến (Eager)
    print("Step 1: Creating Hybrid Candidate Pool...")
    popular_items_list = (
        lf_purchases.group_by("item_id")
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .head(400) 
        .select(pl.col("item_id").cast(pl.Int64))
        .collect()
        .get_column("item_id")
        .to_list()
    )
    popular_items_df = pl.DataFrame({"X_0": popular_items_list})

    # 2. Chuẩn bị danh sách User
    all_users = (
        infer_features_df
        .select(pl.col("X_-1").cast(pl.Int64))
        .unique()
        .collect()
        .get_column("X_-1")
        .to_list()
    )
    print(f"Step 2: Processing {len(all_users):,} users in batches of {batch_size}...")

    all_recommendations = []
    
    # Ép kiểu features sang Int64 và để ở dạng Lazy để join tối ưu
    lazy_features = infer_features_df.with_columns([
        pl.col("X_-1").cast(pl.Int64),
        pl.col("X_0").cast(pl.Int64)
    ]).lazy()

    # Xác định tên cột item chính xác trong user_items
    item_col = "item_id_list" if "item_id_list" in user_items.columns else "item_id"

    for i in range(0, len(all_users), batch_size):
        batch_u = all_users[i : i + batch_size]
        batch_u_df = pl.DataFrame({"X_-1": batch_u})

        # --- NGUỒN A: Popular (400 món) ---
        c_pop = batch_u_df.join(popular_items_df, how="cross")

        # --- NGUỒN B: Bought-together (Cá nhân hóa - Eager) ---
        c_bought = (
            batch_u_df.join(user_items.rename({"customer_id": "X_-1"}), on="X_-1")
            .explode(item_col)
            .cast({item_col: pl.Int64})
            .join(top_item_links, left_on=item_col, right_on="item_id_source")
            .select([pl.col("X_-1"), pl.col("item_id_rec").alias("X_0")])
            .unique()
        )

        # Gộp ứng viên và chuyển sang Lazy để Join Feature
        # c_pop và c_bought đều là DataFrame, concat xong chuyển sang lazy
        candidates_lazy = pl.concat([c_pop, c_bought]).unique().lazy()

        # Join Features & Predict
        batch_data = (
            candidates_lazy
            .join(lazy_features.filter(pl.col("X_-1").is_in(batch_u)), on=["X_-1", "X_0"], how="left")
            .with_columns(pl.all().exclude(["X_-1", "X_0"]).fill_null(0))
            .collect()
        )
        
        # XGBoost Predict
        X_infer = batch_data.select(["X_1", "X_2", "X_3", "X_4", "X_5"]).to_pandas()
        probs = model.predict(xgb.DMatrix(X_infer))
        batch_result = batch_data.with_columns(pl.Series("prob", probs))
        
        # --- CHIẾN THUẬT RERANKING: Ưu tiên nhẹ các món Bought-together ---
        # SỬA LỖI: c_bought đã là DataFrame nên KHÔNG gọi .collect() nữa
        batch_result = batch_result.join(
            c_bought.with_columns(pl.lit(1.15).alias("multiplier")), 
            on=["X_-1", "X_0"], 
            how="left"
        ).with_columns(
            (pl.col("prob") * pl.col("multiplier").fill_null(1.0)).alias("final_score")
        )

        # Lọc món đã mua
        purchased = (
            lf_purchases.filter(pl.col("customer_id").cast(pl.Int64).is_in(batch_u))
            .select([pl.col("customer_id").alias("X_-1").cast(pl.Int64), pl.col("item_id").alias("X_0").cast(pl.Int64)])
            .collect()
        )
        
        top_recs = (
            batch_result.join(purchased, on=["X_-1", "X_0"], how="anti")
            .sort(["X_-1", "final_score"], descending=[False, True])
            .group_by("X_-1")
            .head(top_k)
            .select(["X_-1", "X_0"])
        )
        
        all_recommendations.append(top_recs)
        
        if (i // batch_size) % 10 == 0:
            print(f"   Processed {i + len(batch_u):,} / {len(all_users):,} users...")

    return pl.concat(all_recommendations)

# --- THỰC THI ---
recommendations_warm = run_inference_local_v4(
    model=model_xgb,
    infer_features_df=infer_features_df,
    lf_purchases=full_history,
    top_item_links=top_item_links, 
    user_items=user_items,
    batch_size=2000 
)

Step 1: Creating Hybrid Candidate Pool...
Step 2: Processing 628,069 users in batches of 2000...
   Processed 2,000 / 628,069 users...
   Processed 22,000 / 628,069 users...
   Processed 42,000 / 628,069 users...
   Processed 62,000 / 628,069 users...
   Processed 82,000 / 628,069 users...
   Processed 102,000 / 628,069 users...
   Processed 122,000 / 628,069 users...
   Processed 142,000 / 628,069 users...
   Processed 162,000 / 628,069 users...
   Processed 182,000 / 628,069 users...
   Processed 202,000 / 628,069 users...
   Processed 222,000 / 628,069 users...
   Processed 242,000 / 628,069 users...
   Processed 262,000 / 628,069 users...
   Processed 282,000 / 628,069 users...
   Processed 302,000 / 628,069 users...
   Processed 322,000 / 628,069 users...
   Processed 342,000 / 628,069 users...
   Processed 362,000 / 628,069 users...
   Processed 382,000 / 628,069 users...
   Processed 402,000 / 628,069 users...
   Processed 422,000 / 628,069 users...
   Processed 442,000 / 628,06

In [10]:
# ======================================================================
# 6. COLD START HANDLING (FULL PIPELINE: GT + POPULAR + CO-OCCURRENCE)
# ======================================================================
import polars as pl
import joblib
import os
import numpy as np

print("[1/4] Khởi tạo dữ liệu nền tảng...")

# --- 1. Nạp Ground Truth (.pkl) ---
gt_path = "groundtruth_main.pkl"
if os.path.exists(gt_path):
    gt_raw = joblib.load(gt_path)
    print(f" ✅ Đã nạp Ground Truth: {len(gt_raw):,} users.")
else:
    raise FileNotFoundError(f"❌ Không tìm thấy file {gt_path}!")

# --- 2. Tạo danh sách Popular Items (Candidate Pool) ---
# Giải quyết lỗi NameError: 'popular_items'
popular_items = (
    full_history
    .group_by("item_id")
    .agg(pl.len().alias("cnt"))
    .sort("cnt", descending=True)
    .head(200)
    .select(pl.col("item_id").alias("X_0"))
    .collect()
)
popular_items_list = [int(i) for i in popular_items.get_column("X_0").to_list()[:10]]
print(f" ✅ Đã tạo Candidate Pool (Top 200).")

# --- 3. Tạo bảng lịch sử mua hàng (User-Item History) ---
# Giải quyết lỗi NameError: 'user_items'
user_items = (
    full_history
    .select([
        pl.col("customer_id").cast(pl.Int64), 
        pl.col("item_id").cast(pl.Int64)
    ])
    .drop_nulls()
    .unique() 
    .group_by("customer_id")
    .agg(pl.col("item_id").alias("item_id_list"))
    .collect()
)
print(f" ✅ Đã tạo User-Item History: {user_items.height:,} users.")

# --- 4. Xác định nhóm Cold Users ---
target_users = set(gt_raw.keys()) 
warm_users_list = recommendations_warm.select("X_-1").unique().get_column("X_-1").to_list()
warm_users_set = set(str(u) for u in warm_users_list)
cold_users_ids = list(target_users - warm_users_set)
print(f" ✅ Xác định: {len(warm_users_set):,} Warm users và {len(cold_users_ids):,} Cold users.")

# --- 5. Tính toán Ma trận đồng xuất hiện (Co-occurrence Matrix) ---
print("[2/4] Đang tính toán Ma trận Co-occurrence (Item-Item)...")
user_items_clean = (
    user_items
    .explode("item_id_list")
    .rename({"item_id_list": "item_id"})
    .cast({"item_id": pl.Int64})
)

item_pairs = (
    user_items_clean.join(user_items_clean, on="customer_id", suffix="_right")
    .filter(pl.col("item_id") < pl.col("item_id_right"))
    .group_by(["item_id", "item_id_right"])
    .len()
    .filter(pl.col("len") > 2)
)

top_item_links = (
    pl.concat([
        item_pairs.select([pl.col("item_id"), pl.col("item_id_right"), "len"]),
        item_pairs.select([pl.col("item_id_right").alias("item_id"), pl.col("item_id").alias("item_id_right"), "len"])
    ])
    .sort(["item_id", "len"], descending=[False, True])
    .group_by("item_id")
    .head(10)
    .select([
        pl.col("item_id").alias("item_id_source"), 
        pl.col("item_id_right").alias("item_id_rec")
    ])
)

# --- 6. Sinh gợi ý cho nhóm Cold Start ---
print("[3/4] Đang tạo gợi ý cá nhân hóa cho Cold Users...")
df_cold = pl.DataFrame({"X_-1": [int(u) for u in cold_users_ids]})

# Logic Semi-cold (Dùng Co-occurrence)
batch_semi = (
    df_cold.join(user_items.rename({"customer_id": "X_-1"}), on="X_-1", how="inner")
    .explode("item_id_list")
    .rename({"item_id_list": "item_id"})
    .join(top_item_links, left_on="item_id", right_on="item_id_source", how="inner")
    .filter(pl.col("item_id_rec") != pl.col("item_id"))
    .group_by(["X_-1", "item_id_rec"])
    .len()
    .sort(["X_-1", "len"], descending=[False, True])
    .group_by("X_-1")
    .head(10)
    .select([pl.col("X_-1"), pl.col("item_id_rec").alias("X_0")])
)

# Logic Pure-cold (Dùng Popularity)
batch_pure = (
    df_cold.join(batch_semi.select("X_-1").unique(), on="X_-1", how="anti")
    .with_columns(pl.lit(popular_items_list).alias("X_0"))
    .explode("X_0")
    .select(["X_-1", "X_0"])
)

# --- 7. Kết quả cuối cùng ---
recommendations_cold = pl.concat([batch_semi, batch_pure]).unique()
print(f"✅ [4/4] Hoàn tất! Tổng gợi ý Cold Start: {recommendations_cold.height:,}")

[1/4] Khởi tạo dữ liệu nền tảng...
 ✅ Đã nạp Ground Truth: 391,900 users.
 ✅ Đã tạo Candidate Pool (Top 200).
 ✅ Đã tạo User-Item History: 628,069 users.
 ✅ Xác định: 628,069 Warm users và 391,900 Cold users.
[2/4] Đang tính toán Ma trận Co-occurrence (Item-Item)...
[3/4] Đang tạo gợi ý cá nhân hóa cho Cold Users...
✅ [4/4] Hoàn tất! Tổng gợi ý Cold Start: 3,917,010


In [11]:
# ======================================================================
# 7. FINAL EVALUATION
# ======================================================================
# Nối bảng
recommendations_all = pl.concat([
    recommendations_warm.select([pl.col("X_-1").cast(pl.Int64), pl.col("X_0").cast(pl.Int64)]),
    recommendations_cold.select([pl.col("X_-1").cast(pl.Int64), pl.col("X_0").cast(pl.Int64)])
]).unique(subset=["X_-1", "X_0"])
gt = joblib.load(gt_path)

# Chuyển sang Dict để tính điểm nhanh
pred_df = recommendations_all.group_by("X_-1").agg(pl.col("X_0"))
pred_data = pred_df.to_dict(as_series=False)
pred_dict = dict(zip(pred_data["X_-1"], pred_data["X_0"]))
gt_dict = {int(u): set(map(int, v)) for u, v in gt.items()}

# Tính Precision@10
precisions = []
for u, gt_items in gt_dict.items():
    preds = pred_dict.get(u, [])[:10]
    correct = sum(1 for it in preds if it in gt_items)
    precisions.append(correct / 10)

print(f"🎯 FINAL MEAN PRECISION@10: {np.mean(precisions):.6f}")

🎯 FINAL MEAN PRECISION@10: 0.022038
